In [2]:
from skimage.io import imread, imsave, imshow
import numpy as np
from os.path import join
from os import makedirs
from segmentation_models import Unet

Using TensorFlow backend.
/data3/floraxue/cs282a/assn4env/lib/python3.5/site-packages/classification_models/resnext/__init__.py:4: UserWarning: Current ResNext models are deprecated, use keras.applications ResNeXt models
  warnings.warn('Current ResNext models are deprecated, '


In [1]:
import glob
filelist_x = glob.glob('/data3/floraxue/cs282a/data/dataset_1/raw_crop_image_100/*.tif')
filelist_y = glob.glob('/data3/floraxue/cs282a/data/dataset_1/label_RGB_crop_100/*.tif')

In [4]:
nn =10000
X = np.array([imread(fname) for fname in filelist_x[:nn]])
Y = np.array([imread(fname) for fname in filelist_y[:nn]])
Y = np.sum(Y, axis=3)

In [45]:
nn =10000
X = np.reshape(X, [nn,256,256])
Y = np.reshape(Y, [nn,256,256])

In [46]:
Y.shape

(10000, 256, 256)

In [12]:
!pip install sklearn
from sklearn.model_selection import train_test_split

     |████████████████████████████████| 5.3MB 13.7MB/s eta 0:00:01
  Stored in directory: /home/floraxue/.cache/pip/wheels/76/03/bb/589d421d27431bcd2c6da284d5f2286c8e3b2ea3cf1594c074
Successfully built sklearn


In [ ]:
from segmentation_models import FPN
from segmentation_models.backbones import get_preprocessing
from segmentation_models.losses import bce_jaccard_loss
from segmentation_models.metrics import iou_score


BACKBONE = 'resnet34'
preprocess_input = get_preprocessing(BACKBONE)

# load your data

x_train, x_val, y_train, y_val = train_test_split(X, Y, test_size=0.3, random_state=42)
N = x_train.shape[-1]

# preprocess input
x_train = preprocess_input(x_train)
x_val = preprocess_input(x_val)

# define model
base_model = FPN(BACKBONE, encoder_weights='imagenet')

inp = Input(shape=(None, None, N))
l1 = Conv2D(3, (1, 1))(inp) # map N channels data to 3 channels
out = base_model(l1)

model = Model(inp, out, name=base_model.name, classes=4, activation='softmax')

#model = FPN(BACKBONE, input_shape=(None, 256, 256), classes=4, activation='softmax', encoder_weights=None)
model.compile('Adam', loss='categorical_crossentropy', metrics=['categorical_accuracy'])

# fit model
model.fit(
    x=x_train,
    y=y_train,
    batch_size=32,
    epochs=2,
    validation_data=(x_val, y_val),
)

85524480/85521592 [==============================] - 2s 0us/step


In [ ]:
x_val.shape